In [3]:
import sympy as sy
import numpy as np
from numpy import linalg
import cv2

dir1 = "./images/house1.JPG"
dir2 = "./images/house2.JPG"

img1 = cv2.imread(dir1)
img2 = cv2.imread(dir2)

w1, h1 = img1.shape[:2]
w2, h2 = img2.shape[:2]

print(w1, h1)
print(w2, h2)

1279 1706
1279 1706


In [4]:
def calc_inside_param(F, e, p11, p12, p21, p22, calc_phase="f"):
    """
    F行列と画像のエピポールeからkruppa方程式よりカメラ内部パラメータ行列Aを算出する
    ただし内部パラメータの未知の変数は2個という制限があるため、
    A1 = [[f_u1,  0, c_u1],
          [0,  f_v1, c_v1],
          [0,     0,    1]]
    A2 = [[f_u2,  0, c_u2],
          [0,  f_v2, c_v2],
          [0,     0,    1]]
    のf, cのうち2つしか変数にすることができず、その他の6つの値は埋める必要がある
    今回、f_u1 = f_v1 = f1, f_u2 = f_v2 = f2と仮定して
    与える値を(p11, p12, p21, p22)の4つとしている
    <calc_phase>
    f: f1, f2を求める
    c1: c_u1, c_v1を求める
    c2: c_u2, c_v2を求める
    """

    # a1, a2を求める
    a1, a2 = sy.var('a1, a2')

    # 焦点距離f1, f2を求める
    if calc_phase == "f":
        A1 = sy.Matrix([[a1, 0, p11], [0, a1, p12], [0, 0, 1]])

        A2 = sy.Matrix([[a2, 0, p21],
                        [0, a2, p22],
                        [0, 0, 1]])

        a1_name, a2_name = "f1", "f2"

    # 画像1の中心座標(cu1, cv1)を求める
    elif calc_phase == "c1":
        A1 = sy.Matrix([[p11, 0, a1],
                        [0, p11, a2],
                        [0, 0, 1]])

        A2 = sy.Matrix([[p12, 0, p21],
                        [0, p12, p22],
                        [0, 0, 1]])

        a1_name, a2_name = "cu1", "cv1"

    # 画像2の中心座標(cu2, cv2)を求める
    elif calc_phase == "c2":
        A1 = sy.Matrix([[p11, 0, p21],
                        [0, p11, p22],
                        [0, 0, 1]])

        A2 = sy.Matrix([[p12, 0, a1],
                        [0, p12, a2],
                        [0, 0, 1]])

        a1_name, a2_name = "cu2", "cv2"

    # t
    t = sy.var('t')
    vec_t = sy.Matrix([1, t, 0])

    # e, Fをsympyに変換
    e = sy.Matrix([e[0], e[1], e[2]])
    F = sy.Matrix([[F[0,0],F[0,1],F[0,2]], 
                    [F[1,0],F[1,1],F[1,2]], 
                    [F[2,0],F[2,1],F[2,2]]])

    # eq1 = (e × t)^T * A1 * A1^T * (e1 × t) = 0
    tmp1 = A1.transpose() * (e.cross(vec_t))
    print(tmp1.transpose() * tmp1)
    eq1 = sy.expand((tmp1.transpose() * tmp1)[0])
    print(eq1)

    # eq1からt^0, t^1, t^2の係数を取り出す
    k10 = eq1.coeff(t, 0)
    k11 = eq1.coeff(t, 1)
    k12 = eq1.coeff(t, 2)

    # eq2 = (F^T × t)^T * A2 * A2^T * (F^T × t) = 0
    tmp2 = A2.transpose() * F.transpose() * vec_t
    eq2 = sy.expand((tmp2.transpose() * tmp2)[0])
    print(eq2)
    
    # eq2からt^0, t^1, t^2の係数を取り出す
    k20 = eq2.coeff(t, 0)
    k21 = eq2.coeff(t, 1)
    k22 = eq2.coeff(t, 2)

    # expr1 = k10*k21 - k11*k20
    expr1 = sy.expand(k10*k21 - k11*k20)
    # expr2 = k11*k22 - k21*k12
    expr2 = sy.expand(k11*k22 - k21*k12)
    print("expr1 = {}".format(expr1))
    print("expr2 = {}".format(expr2))

    # expr1 = expr2 = 0を解く(解はa1, a2)
    ans = sy.solve([expr1, expr2], [a1, a2])
    print("({}, {})=\n{}".format(a1_name, a2_name, ans))

    return ans


In [5]:
F = np.array([[-0.00165067, -0.00302917, 0.0224756], [0.0156798, -0.00798849, -0.209402], [0.00316947, 0.146741, -0.0588552]])

In [ ]:
F = np.array([[1, 2, 3], [4, 5, 6], [0.00316947, 0.146741, -0.0588552]])

In [43]:
F = np.array([[-4.64167338e-07,  5.16225960e-06, -1.15368358e-02], [-6.65035697e-06, -1.89664016e-07, -2.48445361e-02], [ 1.37332793e-02,  2.69508065e-02,  1.00000000e+00]])

In [6]:
def calc_epipole(F):
    """ F行列からエピポールeを算出 """

    # Fを特異値分解
    D, V = linalg.eig(np.dot(F, F.T))

    eig_vals_sorted = np.sort(np.absolute(D))
    eig_vecs_sorted = V.T[D.argsort()]
    # Vから固有値最小の固有ベクトルを取り出す
    fvec = eig_vecs_sorted[0]

    return fvec / fvec[2]

e1 = calc_epipole(F)

print(e1)

[38.33338581  3.83334762  1.        ]


In [7]:

# cu1,cv1 = calc_inside_param(F, e1, w1/2, h1/2, w2/2, h2/2, "c1")
# f_res = calc_inside_param(F, e1, w1/2, h1/2, w2/2, h2/2, "f")
f_res = calc_inside_param(F, e1, 0, 0, 0, 0, "f")
f1, f2 = f_res[-1]

Matrix([[1.0*a1**2*t**2 + 1.0*a1**2 + 1469.44846772793*(t - 0.100000235912174)**2]])
1.0*a1**2*t**2 + 1.0*a1**2 + 1469.44846772793*t**2 - 293.890386867152*t + 14.6945540095176
0.0003096721005201*a2**2*t**2 - 3.3673624254e-6*a2**2*t + 1.19005823378e-5*a2**2 + 0.043849197604*t**2 - 0.0094128711824*t + 0.00050515259536
expr1 = -3.3673624254e-6*a1**2*a2**2 - 0.0094128711824*a1**2 + 0.00344798485817078*a2**2 + 0.0101415477028874
expr2 = 3.3673624254e-6*a1**2*a2**2 + 0.0094128711824*a1**2 - 0.0860614878675271*a2**2 + 0.944871488244356
(f1, f2)=
[(-2.30000646050684, -3.40000167534009), (-2.30000646050684, 3.40000167534009), (2.30000646050684, -3.40000167534009), (2.30000646050684, 3.40000167534009)]


In [56]:
points = calc_inside_param(F, e1, f1, f2, 0, 0, "c1")

Matrix([[5.29002971837321*t**2 + 1469.44846772793*(-0.0260869208092607*a1*t + 0.0260869208092607*a2 + t - 0.100000235912174)**2 + 5.29002971837321]])
1.0*a1**2*t**2 - 2.0*a1*a2*t - 76.6667716218163*a1*t**2 + 7.66669524880639*a1*t + 1.0*a2**2 + 76.6667716218163*a2*t - 7.66669524880639*a2 + 1474.73849744631*t**2 - 293.890386867152*t + 19.9845837278908
0.0474290106138946*t**2 - 0.00945179793039968*t + 0.000642723462760156
expr1 = 0.00128544692552031*a1*a2 - 0.00492756491823968*a1 - 0.00945179793039968*a2**2 + 0.0231885213502575*a2 - 2.77555756156289e-17
expr2 = 0.00945179793039968*a1**2 - 0.0948580212277892*a1*a2 - 0.361015063016374*a1 + 3.63622912498416*a2 + 1.77635683940025e-15
(cu1, cv1)=
[(-1.48870225023989e-14, -1.96654170032839e-15), (76.6667716218168, 7.66669524880646), (38.3333858109082 - 0.228452948290566*I, 3.83334762440319 + 2.28452409343421*I), (38.3333858109082 + 0.228452948290566*I, 3.83334762440319 - 2.28452409343421*I)]


In [50]:
f1, f2


(1054.52319749964*I, 947.29995111495*I)

1.71226052963841